# Exploratory data analysis for covid policy data

# 0. Imports

In [1]:
# configure settings
%reset -f
%config InlineBackend.figure_format = 'svg'


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from IPython.display import clear_output
import us
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm.notebook import tqdm

# custom packages
from covid_project.plotting_funcs import plot_counts_with_label
from covid_project.data_utils import load_policy_data, eval_df, check_data

BIG_TEXT   = 18
MED_TEXT   = 14
SMALL_TEXT = 10


# 1. Data Ingestion

In [2]:
path = "./data/covid_policies.csv"
df = load_policy_data(path, force_reload=False)
df.head()

,state_id,policy_level,date,policy_type,start_stop,comments,source,geocoded_state,county,fips_code,total_phases
0,DE,state,2020-07-06T00:00:00.000,Phase 1,stop,Policy_Details: Governor Carney extended Delaw...,sip_submission_form: https://governor.delaware...,"{'type': 'Point', 'coordinates': [-75.52474, 3...",NaN,NaN,NaN
1,MS,county,2020-07-20T00:00:00.000,Outdoor and Recreation,stop,Policy_Details: Executive Order by Governor Ta...,sip_submission_form: https://www.sos.ms.gov/co...,"{'type': 'Point', 'coordinates': [-90.18045, 3...",Sunflower,28133,NaN
2,PR,state,2020-06-15T00:00:00.000,Non-Essential Businesses,stop,Policy_Details: Retail - 50% \nPersonal Care -...,sip_submission_form: https://www.estado.pr.gov...,"{'type': 'Point', 'coordinates': [-51.21156, -...",NaN,NaN,NaN
3,MO,state,2020-06-15T00:00:00.000,Non-Essential Businesses,stop,Policy_Details: All industries: practice socia...,sip_submission_form: https://health.mo.gov/liv...,"{'type': 'Point', 'coordinates': [-92.17785, 3...",NaN,NaN,NaN
4,DE,state,2020-09-05T00:00:00.000,Phase 2,stop,Policy_Details: Governor Carney formally exten...,sip_submission_form: https://governor.delaware...,"{'type': 'Point', 'coordinates': [-75.52474, 3...",NaN,NaN,NaN


# 2 Data evaluation

## 2.1 Overview

In [3]:
eval_df(df)

field             num_nulls  datatypes
--------------  -----------  ---------------------------------
state_id                  0  {<class 'str'>}
policy_level              0  {<class 'str'>}
date                      0  {<class 'str'>}
policy_type               0  {<class 'str'>}
start_stop                0  {<class 'str'>}
comments                  0  {<class 'str'>}
source                    0  {<class 'str'>}
geocoded_state          173  {<class 'float'>, <class 'dict'>}
county                 1706  {<class 'str'>, <class 'float'>}
fips_code              1706  {<class 'str'>, <class 'float'>}
total_phases           2941  {<class 'str'>, <class 'float'>}


drop unneeded policies

In [4]:
df = df.drop(['geocoded_state', 'comments', 'source', 'total_phases'], axis=1)

df.head()

,state_id,policy_level,date,policy_type,start_stop,county,fips_code
0,DE,state,2020-07-06T00:00:00.000,Phase 1,stop,NaN,NaN
1,MS,county,2020-07-20T00:00:00.000,Outdoor and Recreation,stop,Sunflower,28133
2,PR,state,2020-06-15T00:00:00.000,Non-Essential Businesses,stop,NaN,NaN
3,MO,state,2020-06-15T00:00:00.000,Non-Essential Businesses,stop,NaN,NaN
4,DE,state,2020-09-05T00:00:00.000,Phase 2,stop,NaN,NaN


## 2.2 Column-by-column validation

#### 2.2.1 **state_id**

In [5]:
check_data(df['state_id'], "string", name="state_id", check_negs=False)

checking state_id:
datatypes (expect string): {<class 'str'>}
number of nulls: 0



Check which states / territories are in this dataset. 

In [6]:
print("num unique states: ", len(df['state_id'].unique()))
df['state_id'].unique()

num unique states:  56


array(['DE', 'MS', 'PR', 'MO', 'GA', 'NM', 'CA', 'MN', 'WI', 'PA', 'NE',
       'WY', 'LA', 'RI', 'UT', 'IN', 'ID', 'ME', 'TX', 'OK', 'DC', 'OH',
       'IA', 'AK', 'FL', 'VT', 'CO', 'NY', 'NV', 'MA', 'NC', 'KY', 'OR',
       'GU', 'AZ', 'MT', 'IL', 'ND', 'WV', 'KS', 'AR', 'MI', 'NJ', 'CT',
       'AL', 'TN', 'VI', 'HI', 'NH', 'MD', 'WA', 'VA', 'SC', 'MP', 'SD',
       'PW'], dtype=object)

To make this column compatable with 'state' in the case / death dataset, we'll do the following:

- convert ID's to full names

- restrict to the 50 states

- rename the column to 'state'

In [7]:
abbr = [elem.abbr for elem in us.states.STATES]
df = df.drop(df[~df['state_id'].isin(abbr)].index)
df.replace(to_replace=us.states.mapping('abbr', 'name'), inplace=True)
df.rename(columns={'state_id': 'state'}, inplace=True)

print("num unique states: ", len(df['state'].unique()))
df['state'].unique()

num unique states:  50


array(['Delaware', 'Mississippi', 'Missouri', 'Georgia', 'New Mexico',
       'California', 'Minnesota', 'Wisconsin', 'Pennsylvania', 'Nebraska',
       'Wyoming', 'Louisiana', 'Rhode Island', 'Utah', 'Indiana', 'Idaho',
       'Maine', 'Texas', 'Oklahoma', 'Ohio', 'Iowa', 'Alaska', 'Florida',
       'Vermont', 'Colorado', 'New York', 'Nevada', 'Massachusetts',
       'North Carolina', 'Kentucky', 'Oregon', 'Arizona', 'Montana',
       'Illinois', 'North Dakota', 'West Virginia', 'Kansas', 'Arkansas',
       'Michigan', 'New Jersey', 'Connecticut', 'Alabama', 'Tennessee',
       'Hawaii', 'New Hampshire', 'Maryland', 'Washington', 'Virginia',
       'South Carolina', 'South Dakota'], dtype=object)

#### 2.2.2 **county**

In [8]:
check_data(df['county'], "string", name="county", check_negs=False)

checking county:
datatypes (expect string): {<class 'str'>, <class 'float'>}
number of nulls: 1619



Nulls in the county field likely refer to statewide policies. Let's verify that there aren't any counties with 'null' that aren't statewide policies.  

In [9]:
df[(df['county'].isnull()) & (df['policy_level']!='state')]

,state,policy_level,date,policy_type,start_stop,county,fips_code


We can safely convert these nulls to 'statewide', then convert all county names to lowercase

In [10]:
df.fillna(value={'county': 'statewide'}, inplace=True)
df['county'] = df['county'].str.lower()

Check that all the counties in df2 are also present in df

In [11]:
case_df = pd.read_csv("./data/covid_timeseries_cleaned.csv")

locs = case_df['county'].unique()
mismatches = [county for county in df['county'][df['county']!='statewide'].unique() 
              if county not in locs]
print(mismatches)

['alameda county', 'york county', 'linn county', 'zapata county', 'brown county', 'webster county', 'el paso county', 'leon county', 'coryell county', 'san patricio county', 'yolo county', 'sutter county', 'cass county', 'monroe county', 'decatur county', 'broomfield county', 'swisher county', 'stoddard county', 'aransas county', 'beaver county', 'kaufman county', 'osceola county', 'hays county', 'erath county', 'jefferson county', 'cobb county', 'sumter county', 'gaston county', 'salt lake county', 'val verde county', 'dallas county', 'mecklenburg county', 'willacy county', 'lincoln county', 'navarro county', 'atchison county', 'chambers county', 'lebanon county', 'young county', 'montgomery county', 'refugio county', 'boone county', 'washington county', 'placer county', 'cumberland county', 'st. lucie county', 'andrews county', 'nevada county', 'jeff davis county', 'shelby county', 'wasatch county', 'jackson county', 'starr county', 'northampton county', 'hillsborough county', 'crawf

There are some slight differences in location names. For example, frio county in the policy dataset is written as frio in the case dataset. Remove the modifiers at the end of the location names so they match the case dataset.

In [12]:
county_match    = re.compile(" county$")
munici_match    = re.compile(" municipality$")
city_match      = re.compile(" city$")
Borough_match   = re.compile(" borough$")

df['county'].replace(to_replace= county_match, value='', inplace=True)
df['county'].replace(to_replace= munici_match, value='', inplace=True)
df['county'].replace(to_replace=   city_match, value='', inplace=True)
df['county'].replace(to_replace=Borough_match, value='', inplace=True)

In [13]:
locs = case_df['county'].unique()
mismatches = [county for county in df['county'][df['county']!='statewide'].unique() 
              if county not in locs]
print(mismatches)

[]


In [14]:
check_data(df['county'], "string", name="county", check_negs=False)

checking county:
datatypes (expect string): {<class 'str'>}
number of nulls: 0



#### 2.2.3 **fips_code**

In [19]:
df['fips_code'] = df['fips_code'].apply(lambda x: float(x))

In [20]:
check_data(df['fips_code'], "integer", name="fips_code")

checking fips_code:
datatypes (expect integer): {<class 'numpy.float64'>}
number of nulls: 1619
number of negative values: 0



These nulls in fips_code should all refer to statewide polices- first check that there are no nulls in fips_code that are not concurrent with a statewide policy. 

In [21]:
df[(df['fips_code'].isnull()) & (df['policy_level'] != 'state')]

,state,policy_level,date,policy_type,start_stop,county,fips_code


All of these null values are statewide policies. Let's replace these nulls with the 2 digit fips code for the state

In [22]:
for index, data in df.iterrows(): 
    if data.policy_level == 'state':
        df.loc[index, 'fips_code'] = np.int64(us.states.lookup(data.state).fips)

Next, we'll ensure that all datapoints are integers then convert to ints

In [23]:
check_data(df['fips_code'], check_ints=True)

all decimal components zero? (expect true) True


In [24]:
df['fips_code'] = df['fips_code'].astype(np.int64)
check_data(df['fips_code'], "integer", name="fips_code")

checking fips_code:
datatypes (expect integer): {<class 'numpy.int64'>}
number of nulls: 0
number of negative values: 0



#### 2.2.4 **policy_level**

In [25]:
check_data(df['policy_level'], "string", name="policy_level", check_negs=False)

checking policy_level:
datatypes (expect string): {<class 'str'>}
number of nulls: 0



In [26]:
df['policy_level'].unique()

array(['state', 'county'], dtype=object)

#### 2.2.5 **date**

In [27]:
check_data(df['date'], "datetime", name="date", check_negs=False)

checking date:
datatypes (expect datetime): {<class 'str'>}
number of nulls: 0



convert these dates to strings like in case_data

Note that there was a typo in the year field in one of the records (0020 instead of 2020 - caused an out of bounds exception for datetime)

In [30]:
bad_mask = df['date'].str.contains('0020')
df.loc[bad_mask, 'date'] = ['2020' + elem[4:] for elem in df.loc[bad_mask, 'date'].values]

In [31]:
df['date'] = pd.to_datetime(df['date'].str[:10], format='%Y-%m-%d')

Drop any policies that may have been implemented before the start of the case dataset.

In [32]:
df = df.drop(df[(df['date']<min(case_df['date'])) | (df['date']>max(case_df['date']))].index)

In [33]:
check_data(df['date'], "datetime", name="date", check_negs=False)

checking date:
datatypes (expect datetime): {<class 'numpy.datetime64'>}
number of nulls: 0



#### 2.2.6 **policy_type**

In [34]:
check_data(df['policy_type'], "string", name="policy_type", check_negs=False)

checking policy_type:
datatypes (expect string): {<class 'str'>}
number of nulls: 0



We will depend on policy_type quite a bit later on, so let's investigate what kind of policies are in this dataset

In [35]:
print(np.sort((df['policy_type'].unique())))

['Agriculture' 'Allow Audio Only Telehealth'
 'Allow Expand Medicaid Telehealth Coverage' 'Alternative Care Facilities'
 'Bars' 'Childcare (K-12)' 'Colleges & Universities' 'Construction'
 'Day Care' 'Day camps/overnight camps' 'Education' 'Election'
 'Entertainment' 'Executive Order' 'Food and Drink' 'Gatherings'
 'Graduation' 'Graduation Ceremony guidelines' 'Gyms' 'Health Risk Status'
 'Houses of Worship'
 'Mandate Face Mask Use By All Individuals In Public Facing Businesses'
 'Mandate Face Mask Use By All Individuals In Public Spaces'
 'Manufacturing' 'Mask Requirement' 'Medical'
 'Modify Medicaid Requirements With 1135 Waivers Date Of CMS Approval'
 'New Phase' 'Non-Essential Businesses' 'Nursing Home Visitations'
 'Nursing Homes' 'Outdoor and Recreation' 'Personal Care' 'Phase 1'
 'Phase 2' 'Phase 2, 3' 'Phase 3' 'Phase 3 Step 2' 'Phase 4' 'Phase 5'
 'Public Gatherings' 'Public Health Advisory System' 'Quarantine'
 'Reopened ACA Enrollment Using a Special Enrollment Period'
 'Res

Some of these names are a bit too long. Rename any policy type longer than 40 characters

In [36]:
long_names = [policy for policy in df['policy_type'].unique() if len(policy)>40]
for i in long_names: 
    print(i)

Stop Enforcement Of Evictions Overall Or Due To Covid Related Issues
Suspended Elective Medical Dental Procedures
Modify Medicaid Requirements With 1135 Waivers Date Of CMS Approval
Mandate Face Mask Use By All Individuals In Public Facing Businesses
Stop Initiation Of Evictions Overall Or Due To Covid Related Issues
Allow Expand Medicaid Telehealth Coverage
Mandate Face Mask Use By All Individuals In Public Spaces
Reopened ACA Enrollment Using a Special Enrollment Period


In [37]:
policy_replacements_dict = {
    'Stop Initiation Of Evictions Overall Or Due To Covid Related Issues': 'Stop Initiation Of Evictions',
    'Modify Medicaid Requirements With 1135 Waivers Date Of CMS Approval': 'Modify Medicaid Requirements', 
    'Stop Enforcement Of Evictions Overall Or Due To Covid Related Issues': 'Stop Enforcement Of Evictions', 
    'Mandate Face Mask Use By All Individuals In Public Facing Businesses':  'Mandate Face Masks In Businesses', 
    'Mandate Face Mask Use By All Individuals In Public Spaces': 'Mandate Face Masks In Public Spaces', 
    'Reopened ACA Enrollment Using a Special Enrollment Period': 'ACA Special Enrollment Period', 
    'Suspended Elective Medical Dental Procedures': 'Suspend Elective Dental Procedures', 
    'Allow Expand Medicaid Telehealth Coverage': 'Expand Medicaid Telehealth Coverage', 
    'Renter Grace Period Or Use Of Security Deposit To Pay Rent': 'Grace Period / Security Deposit for Rent'
}

for key in policy_replacements_dict.keys():
    df['policy_type'].replace(to_replace=key, value=policy_replacements_dict[key], inplace=True)

long_names = [policy for policy in df['policy_type'].unique() if len(policy)>40]
if len(long_names) == 0: 
    print("no more long names")

no more long names


Aditionally, let's convert everything to lowercase for consistency

In [38]:
df['policy_type'] = df['policy_type'].str.lower()

Some of these policies are also vague and non-specific, so we'll drop these as well 

In [39]:
policies_drop = ["phase 1", "phase 2", "phase 3", "phase 4", "phase 5", "new phase"]
df = df.drop(df[df['policy_type'].isin(policies_drop)].index)

#### 2.2.7 **start_stop**

In [40]:
check_data(df['start_stop'], "string", name="start_stop", check_negs=False)

checking start_stop:
datatypes (expect string): {<class 'str'>}
number of nulls: 0



In [41]:
df['start_stop'].unique()

array(['stop', 'start'], dtype=object)

In [42]:
df.to_csv("./data/covid_policy_data_cleaned.csv")